In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot

%matplotlib inline

# Preparação dos dados

## Funções de apoio

In [2]:
def transform_to_year_df(df, origin='quarter'):  
    if origin == 'quarter':
        _df = df.loc[[index[len(index) - 2:len(index)] == "Q1" for index in df.index]]
        _df.index = range(int(_df.index[0][:4]), int(_df.index[len(_df) - 1][:4]) + 1)
    elif origin == 'monthly':
        _df = df.loc[[index.month == 1 for index in df.index]]
        _df.index = [index.year for index in _df.index]
    
    return _df

## Fontes de dados

* Formação bruta de capital fixo
* Alojamento local
* Golden Visas
* Taxas de juro para crédito habitação
* Índice de preços das casas
* Rendimentos anuais
* Principais agregados do PIB
* Noites passadas por turistas

In [3]:
#  Formação bruta de capital fixo
## Fonte: Eurostat
## Indice 100=2005

fbcf_df = pd.read_excel('data/namq_10_an6.xls', skiprows=10, skipfooter=3, index_col=0)
fbcf_df.columns = ['fbcf', 'construction']

fbcf_df = transform_to_year_df(fbcf_df)
fbcf_df.tail()

,fbcf,construction
2015,70.2,52.5
2016,69.8,50.3
2017,77.9,56.5
2018,82.8,59.2
2019,91.5,66.5


In [4]:
#  Estatisticas alojamento local
## Fonte: https://travelbi.turismodeportugal.pt/pt-pt/Paginas/PowerBI/rnal-registo-nacional-de-alojamento-local.aspx
## Dados: Nº Registos alojamento local

alojamento_df = pd.DataFrame(
    data=[1088, 1184, 1282, 2002, 3958, 5784, 7564, 9516, 11985, 16729, 27883, 39304, 57723, 81054, 92045],
    index=range(2005, 2020),
    columns=['alojamento_local']
)

alojamento_df.tail()

,alojamento_local
2015,27883
2016,39304
2017,57723
2018,81054
2019,92045


In [5]:
# Golden Visas
## Fonte: https://www.sef.pt/pt/pages/conteudo-detalhe.aspx?nID=93
## Dados: Autorizações de residencia para investimentos

golden_df = pd.DataFrame(
    data=[1, 1, 1, 1, 1, 1, 1, 2, 494, 1526, 766, 1414, 1451, 1409, 1245],
    index=range(2005, 2020),
    columns=['golden_visas']
)

golden_df.tail()

,golden_visas
2015,766
2016,1414
2017,1451
2018,1409
2019,1245


In [6]:
# Taxa de juro crédito habitação
## Fonte: Banco de Portugal
## Dados: Valor da taxa de juro por mês
juro_df = pd.read_csv('data/bportugal - taxa de juro.csv', index_col=0)

juro_df.index = pd.to_datetime(juro_df.index)
juro_df = juro_df.rolling(12).mean()
juro_df = transform_to_year_df(juro_df, 'monthly')
juro_df.columns = ['juro']

juro_df.tail()

,juro
2016,2.3225
2017,1.9225
2018,1.6250
2019,1.4050
2020,1.1925


In [7]:
# Índice preços das casas
## Fonte: Eurostat
## 2005 = 100

housing_df = pd.read_excel('data/prc_hpi_q.xls', skiprows=8, skipfooter=3, index_col=0).T

housing_df.columns = ['house_prices', 'new_house_prices', 'old_house_prices']
housing_df.house_prices = pd.to_numeric(housing_df.house_prices, errors='coerce')
housing_df.new_house_prices = pd.to_numeric(housing_df.new_house_prices, errors='coerce')
housing_df.old_house_prices = pd.to_numeric(housing_df.old_house_prices, errors='coerce')

housing_df = transform_to_year_df(housing_df)
housing_df.tail()

,house_prices,new_house_prices,old_house_prices
2015,96.94,96.80,97.01
2016,103.67,101.34,104.66
2017,111.89,105.58,114.34
2018,125.58,115.80,129.18
2019,137.14,122.76,142.08


In [8]:
# Rendimentos Anuais
## Fonte: Eurostat
## Dados: Euros

earnings_df = pd.read_excel('data/earn_nt_net.xls', skiprows=10, skipfooter=3, index_col=0).T
earnings_df.columns = ['earnings_single', 'earnings_couple']
earnings_df.index = [int(index) for index in earnings_df.index]

earnings_df.tail()

,earnings_single,earnings_couple
2014,12629.91,26026.83
2015,12467.22,26525.94
2016,12807.39,26945.62
2017,13051.15,27302.30
2018,13469.59,28139.19


In [9]:
# Principais agregados do PIB
## Fonte: Eurostat
## Chain linked volumes, index 2005=100

gdp_df = pd.read_excel('data/namq_10_gdp.xls', skiprows=10, skipfooter=3, index_col=0).T
gdp_df.columns = ['gdp', 'G', 'C', 'I', 'EX', 'IX']

gdp_df = transform_to_year_df(gdp_df)
gdp_df.tail()

,gdp,G,C,I,EX,IX
2015,98.4,92.3,97.7,70.2,152.7,118.3
2016,99.7,93.8,100.5,69.8,154.5,124.0
2017,103.3,93.7,102.9,77.9,171.4,132.9
2018,106.0,94.4,105.2,82.8,180.6,143.0
2019,108.6,95.3,107.9,91.5,187.7,153.1


In [10]:
# Noites passadas em accomodações turisticas
## Fonte: Eurostat
## Dados: Número

tourism_df = pd.read_excel('data/tour_occ_nim.xls', skiprows=10, skipfooter=3, index_col=0).T

# Make 12 month rolling average
tourism_df.columns = ['tourism']
tourism_df.tourism = pd.to_numeric(tourism_df.tourism, errors='coerce')
tourism_df = tourism_df.rolling(12).mean()

# Dezembro enquanto representante para um ano
tourism_df = tourism_df[[index[len(index) - 2:len(index)] == '12' for index in tourism_df.index]]
tourism_df.index = [int(index[:4]) for index in tourism_df.index]

tourism_df.head()

,tourism
2005,1.448034e+06
2006,1.492206e+06
2007,1.549430e+06
2008,1.554549e+06
2009,1.542823e+06


## Criar dataframe final

* Juntar as fontes todas
* Criar variáveis logaritmizadas
* Indexar as variáveis a 2008

In [11]:
full_df = (housing_df[['house_prices']]
    .merge(tourism_df['tourism'], how='left', left_index=True, right_index=True)
    .merge(gdp_df[['gdp', 'C', 'I']], how='left', left_index=True, right_index=True)
    .merge(earnings_df[['earnings_single', 'earnings_couple']], how='left', left_index=True, right_index=True)
    .merge(fbcf_df[['fbcf', 'construction']], how='left', left_index=True, right_index=True)
    .merge(alojamento_df, how='left', left_index=True, right_index=True)
    .merge(golden_df, how='left', left_index=True, right_index=True)
    .merge(juro_df, how='left', left_index=True, right_index=True)
)

In [12]:
full_df['log_juro'] = np.log(full_df['juro'])
full_df['log_alojamento_local'] = np.log(full_df['alojamento_local'])
full_df['log_house_prices'] = np.log(full_df['house_prices'])

In [13]:
full_df.to_csv('data/full_data.csv')

In [14]:
INDEX_BASE_YEAR = 2008

for column in full_df.columns:
    full_df[column] = full_df[column] / full_df[column].loc[INDEX_BASE_YEAR] * 100

In [15]:
full_df.head()

,house_prices,tourism,gdp,C,I,earnings_single,earnings_couple,fbcf,construction,alojamento_local,golden_visas,juro,log_juro,log_alojamento_local,log_house_prices
2005,NaN,93.148216,94.871795,94.512772,96.175908,90.337256,90.062008,96.175908,108.974359,54.345654,100.0,71.421242,78.746316,91.978250,NaN
2006,NaN,95.989678,95.726496,95.837275,95.506692,98.678020,98.186043,95.506692,105.235043,59.140859,100.0,69.676757,77.184784,93.090569,NaN
2007,NaN,99.670724,98.195632,97.256386,97.036329,99.195395,99.098680,97.036329,104.166667,64.035964,100.0,83.564221,88.661662,94.136660,NaN
2008,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,100.000000,100.000000,100.000000,100.000000
2009,96.740822,99.245719,95.631529,97.161779,90.439771,100.857247,101.133837,90.439771,90.705128,197.702298,100.0,109.594664,105.785431,108.966074,99.294023


In [16]:
full_df.to_csv('data/full_data_index.csv')